In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn.functional as F
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn as nn
import ipywidgets as widgets
from IPython.display import display
import csv
import ipywidgets as widgets
from jetbot import Robot
import numpy as np
from datetime import datetime
from jetbot import Camera, bgr8_to_jpeg
import torch
import cv2
import traitlets
import numpy as np
from jetbot import Camera, bgr8_to_jpeg
from torchvision import models, transforms
from ipywidgets import widgets
from IPython.display import display
import time
import torch
import cv2
import traitlets
import numpy as np
from jetbot import Camera, bgr8_to_jpeg
from torchvision import models, transforms
from ipywidgets import widgets
from IPython.display import display
import time
import ipywidgets

In [3]:
# 初始化相機
camera = Camera.instance(width=224, height=224,fps=5)

# 設定 widget 大小
widget_width = 224
widget_height = 224

# 建立顯示影像的 widget 和 x, y 控制滑桿
image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
#target_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
#x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
#y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

# 顯示 widget
#display(widgets.HBox([image_widget]))

# 啟動相機影像的即時處理和推論
time.sleep(1)  # 確保相機啟動完成
#traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
#traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=process_and_predict)


In [11]:
camera.stop()

In [8]:
import time

# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 加載模型
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("lane_detection_model.pth"))
model = model.to(device)
model.eval()  # 將模型設定為推論模式

def execute(change):
    start_time = time.time()  # 開始計時
    image = change['new']
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4691, 0.4032, 0.4579], [0.1740, 0.1485, 0.1688])
    ])
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)
        x, y = output[0].cpu().numpy()
    
    end_time = time.time()  # 結束計時
    execution_time = end_time - start_time  # 計算間隔時間
    print(f'Execution time: {execution_time:.4f} seconds')

execute({'new': camera.value})


cuda
Execution time: 0.1626 seconds


In [9]:
camera.observe(execute, names='value')

Execution time: 0.2259 seconds
Execution time: 0.1684 seconds
Execution time: 0.2095 seconds
Execution time: 0.1891 seconds
Execution time: 0.2377 seconds
Execution time: 0.1834 seconds
Execution time: 0.1851 seconds
Execution time: 0.1591 seconds
Execution time: 0.2052 seconds
Execution time: 0.1521 seconds
Execution time: 0.2150 seconds
Execution time: 0.1701 seconds
Execution time: 0.2230 seconds
Execution time: 0.2184 seconds
Execution time: 0.2441 seconds
Execution time: 0.1811 seconds
Execution time: 0.1664 seconds
Execution time: 0.1198 seconds
Execution time: 0.1388 seconds
Execution time: 0.1230 seconds
Execution time: 0.1311 seconds
Execution time: 0.1370 seconds
Execution time: 0.1204 seconds
Execution time: 0.1178 seconds
Execution time: 0.0963 seconds
Execution time: 0.1113 seconds
Execution time: 0.0899 seconds
Execution time: 0.1078 seconds
Execution time: 0.1138 seconds
Execution time: 0.1180 seconds
Execution time: 0.1187 seconds
Execution time: 0.1152 seconds
Executio

In [10]:
camera.unobserve(execute, names='value')

Execution time: 0.2160 seconds


# 測試秒數TensorRT

In [13]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('resnet34_final_trt.pth'))
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import torch
device = torch.device('cuda')
mean = torch.Tensor([0.4691, 0.4032, 0.4579]).cuda().half()
std = torch.Tensor([0.1740, 0.1485, 0.1688]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def execute(change):
    start_time = time.time()  # 開始計時
    global angle, angle_last,integral
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    end_time = time.time()  # 結束計時
    execution_time = end_time - start_time  # 計算間隔時間
    print(f'Execution time: {execution_time:.4f} seconds')
    
execute({'new': camera.value})

Execution time: 0.1861 seconds
